In [1]:
import numpy as np
np.set_printoptions(suppress=True)

In [2]:
from robot_model import KR6R900sixx

In [3]:
agilus = KR6R900sixx(interact=False)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [4]:
agilus.q = [0.4,-np.pi/2 + 1, np.pi/2, 1, 1, 0]

In [5]:
agilus.q

array([ 0.4       , -0.57079633,  1.57079633,  1.        ,  1.        ,
        0.        ])

In [6]:
class DenavitHartenbergFrame:
    def __init__(self, a, alpha, d, theta):
        self.a = a
        self.alpha = alpha
        self.d = d
        self.theta = theta
        
    def __repr__(self):
        return 'DH(a={}, alpha={}, d={}, theta={})'.format(self.a, self.alpha, self.d, self.theta)

In [7]:
dh_params = [DenavitHartenbergFrame(0.025, -np.pi/2,  0.4, 0),
             DenavitHartenbergFrame(0.455,      0.0,  0.0, 0),
             DenavitHartenbergFrame(0.035, -np.pi/2,  0.0, -np.pi/2),
             DenavitHartenbergFrame(0.0  ,  np.pi/2, 0.42, 0),
             DenavitHartenbergFrame(0.0  , -np.pi/2,  0.0, 0),
             DenavitHartenbergFrame(0.0  ,      0.0, 0.08, np.pi)]

In [8]:
class DenavitHartenbergFKSolver:
    def __init__(self, dh_params, rotation_directions=None):
        self._dh_params = dh_params
        self._rotation_directions = rotation_directions
        
    def _dh_transform(self, a, alpha, d, theta):
        ct = np.cos(theta)
        st = np.sin(theta)
        ca = np.cos(alpha)
        sa = np.sin(alpha)
        return np.array([[ct, -st * ca,  st * sa, a * ct],
                         [st,  ct * ca, -ct * sa, a * st],
                         [0.0,      sa,       ca, d],
                         [0.0,     0.0,      0.0, 1.0]])
    
    @property
    def dh_params(self):
        return self._dh_params
    
    def solve(self, q):
        if self._rotation_directions is not None:
            q *= self._rotation_directions
        intermediate_link_transforms = []
        for dhi, qi in zip(self._dh_params, q):
            intermediate_link_transforms.append(
                self._dh_transform(dhi.a, dhi.alpha, dhi.d, (dhi.theta + qi)))
        link_transforms = [intermediate_link_transforms[0]] # T^0_1
        trf_0_i = intermediate_link_transforms[0]
        for i in range(1, 6):
            trf_0_i = trf_0_i.dot(intermediate_link_transforms[i])
            link_transforms.append(trf_0_i)
        return (link_transforms[-1], link_transforms, intermediate_link_transforms)
        

In [9]:
fk_solver = DenavitHartenbergFKSolver(dh_params, 
                                      rotation_directions=[-1.,1.,1.,-1.,1.,-1.])
trf, _, _ = fk_solver.solve(agilus.q)
trf

array([[-0.8220643 , -0.44177518, -0.3592283 ,  0.58307443],
       [-0.14605091,  0.77338816, -0.61687916, -0.30802059],
       [ 0.55034481, -0.45464871, -0.70029646,  0.2553066 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [10]:
fk_solver = DenavitHartenbergFKSolver(dh_params)

In [11]:
ee_debug_trf = agilus._ee_debug_call_twice()
ee_debug_trf

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [12]:
np.allclose(trf, ee_debug_trf)

False

In [19]:
class KR6R900sixxAnalyticalIK:
    def __init__(self, fk_solver):
        self._fk_solver = fk_solver
        
    def solve(self, trf, front=1, elbow=1, wrist=1):
        ne = trf[:3,0]
        se = trf[:3,1]
        ae = trf[:3,2]
        pe = trf[:3,3]
        
        d1 = self._fk_solver.dh_params[0].d
        d4 = self._fk_solver.dh_params[3].d
        d6 = self._fk_solver.dh_params[5].d
        
        a1 = self._fk_solver.dh_params[0].a
        a2 = self._fk_solver.dh_params[1].a
        a3 = self._fk_solver.dh_params[2].a
        
        l3 = np.sqrt(a3 * a3 + d4 * d4)
        
        pw = pe - d6 * ae
        
        q1 = np.arctan2(front * pw[1], front*pw[0])
        
        pty = - (pw[2] - d1)
        ptx = front * (np.sqrt(pw[0] * pw[0] + pw[1] * pw[1]) - front * a1)
        
        c3 = (ptx * ptx + pty * pty - a2 * a2 - l3 * l3 ) / (2 * a2 * l3)
        s3 = elbow * np.sqrt(1.0 - c3 * c3)
        psi3 = np.arctan2(s3, c3)
        offset3 = np.arctan2(a3, d4)
        q3 = psi3 + offset3
        
        den2 = (ptx * ptx + pty * pty)
        c2 = (ptx * (a2 + l3 * c3) + pty * l3 * s3) / den2
        s2 = (pty * (a2 + l3 * c3) - ptx * l3 * s3) / den2
        q2 = np.arctan2(s2, c2)
        
        _, trfs, _ = self._fk_solver.solve(np.array([q1, q2, q3, 0.0, 0.0, 0.0]))
        
        rot_0_3 = trfs[2][:3,:3]
        rot_3_6 = rot_0_3.T.dot(trf[:3,:3])
        
        q4 = np.arctan2(wrist * rot_3_6[1,2], wrist * rot_3_6[0,2])
        q5 = -np.arctan2(wrist * np.sqrt(rot_3_6[0,2] * rot_3_6[0,2] + rot_3_6[1,2] * rot_3_6[1,2]), rot_3_6[2,2])
        q6 = np.arctan2(wrist * rot_3_6[2,1], -wrist * rot_3_6[2,0]) + self._fk_solver.dh_params[-1].theta
        
        return np.array([q1, q2, q3, q4, q5, q6])

In [22]:
fk_solver = DenavitHartenbergFKSolver(dh_params)
trf, _, _ = fk_solver.solve(np.array([1,1,1,1,1,1]))
ik_solver = KR6R900sixxAnalyticalIK(fk_solver)
q = ik_solver.solve(trf, wrist=-1)
print(q)
trf2, _, _ = fk_solver.solve(q)

[1. 1. 1. 1. 1. 1.]


In [15]:
np.allclose(trf, trf2)

False

In [16]:
trf

array([[-0.21667169,  0.26112258, -0.94067443, -0.0061594 ],
       [ 0.95867353,  0.23890322, -0.1545002 ,  0.09524838],
       [ 0.18438665, -0.93527549, -0.30209487, -0.40350695],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [17]:
trf2

array([[ 0.21667169, -0.26112258, -0.94067443, -0.0061594 ],
       [-0.95867353, -0.23890322, -0.1545002 ,  0.09524838],
       [-0.18438665,  0.93527549, -0.30209487, -0.40350695],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [18]:
trf

array([[-0.21667169,  0.26112258, -0.94067443, -0.0061594 ],
       [ 0.95867353,  0.23890322, -0.1545002 ,  0.09524838],
       [ 0.18438665, -0.93527549, -0.30209487, -0.40350695],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])